# 🏥 Clinical Triage Router

**A Specialized SLM Agent (≤3B) for Patient Intake Classification**

---

## 📋 Overview

| Phase | Description |
|-------|-------------|
| **Phase 1** | Load pre-generated training data (or regenerate via API) |
| **Phase 2** | Fine-tune Qwen2.5-1.5B with LoRA (4-bit quantized) |
| **Phase 3** | Agentic loop with self-correction |
| **Phase 4** | Evaluation with JSON validity & routing accuracy |

### Key Features
- ✅ **Zero-cost**: Runs entirely on Colab Free Tier (T4 GPU)
- ✅ **Small Model**: Qwen2.5-1.5B (1.5B parameters, 4-bit)
- ✅ **Self-Correction**: 3-retry loop with error context injection
- ✅ **Strict Schema**: Pydantic validation for JSON tool calls

---

**Author**: Adarsh P  
**Runtime**: Google Colab T4 GPU (Free Tier)

## 🔧 Setup & Installation

In [ ]:
# Install dependencies (optimized for Colab T4 GPU)
!pip install -q unsloth
!pip install -q --no-deps xformers trl peft accelerate bitsandbytes
!pip install -q google-generativeai pydantic datasets nest_asyncio

In [ ]:
# Clone the repository
!git clone https://github.com/forkadarshp/clinic-triage.git
%cd clinic-triage

In [ ]:
# Verify imports and configuration
from src import config
from src.schemas import ToolName, parse_triage_output, get_mock_response

print("📦 Configuration:")
print(f"   Model: {config.MODEL_NAME}")
print(f"   Tools: {[t.value for t in ToolName]}")
print(f"   Training Examples: {config.NUM_TRAINING_EXAMPLES}")
print("\n✅ All imports successful!")

---
## 📋 Phase 1: Training Data

> **Default Behavior**: Uses pre-generated training data (`data/train.jsonl`) for fast, reliable execution.  
> **To regenerate**: Set `REGENERATE_DATA = True` below (requires API key in Colab Secrets).

In [ ]:
import os
from google.colab import userdata

# Load API keys from Colab Secrets (only needed if REGENERATE_DATA = True)
try:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    print("✅ GOOGLE_API_KEY loaded from Colab Secrets")
except:
    print("ℹ️ GOOGLE_API_KEY not found (not needed for pre-generated data)")

In [ ]:
from src.data_generator import generate_training_data, load_training_data

# ╔═══════════════════════════════════════════════════════════════════╗
# ║  CONFIGURATION FLAG                                               ║
# ║  Set to True to regenerate training data via Gemini API           ║
# ║  Set to False (default) to use pre-generated data/train.jsonl     ║
# ╚═══════════════════════════════════════════════════════════════════╝
REGENERATE_DATA = False

# Load or generate training examples
examples = generate_training_data(
    num_examples=config.NUM_TRAINING_EXAMPLES,
    force_use_existing=not REGENERATE_DATA
)

print(f"\n📊 Loaded {len(examples)} training examples")
print(f"\n📝 Sample Example:")
print(f"   Query: {examples[0]['query'][:80]}...")
print(f"   Tool: {examples[0]['tool']}")
print(f"   Args: {examples[0]['arguments']}")

### 1.2 Load Model (4-bit Quantized)

Using Unsloth's pre-quantized Qwen2.5-1.5B for memory efficiency on T4 GPU.

In [ ]:
from src.trainer import load_model_and_tokenizer, prepare_model_for_training

# Load pre-quantized model (fits in ~6GB VRAM)
model, tokenizer = load_model_and_tokenizer()
print(f"✅ Loaded: {config.MODEL_NAME}")
print(f"   Quantization: 4-bit")
print(f"   Max Sequence Length: {config.MAX_SEQ_LENGTH}")

# Add LoRA adapters for efficient fine-tuning
model = prepare_model_for_training(model)
print(f"\n✅ LoRA adapters attached")
print(f"   Rank (r): {config.LORA_R}")
print(f"   Alpha: {config.LORA_ALPHA}")

### 1.3 Fine-Tune

Training the model to output structured JSON tool calls.

In [ ]:
from src.trainer import prepare_dataset, train

# Prepare dataset with instruction-output pairs
dataset = prepare_dataset(tokenizer)
print(f"📊 Training set: {len(dataset)} examples")

# Train with SFT (Supervised Fine-Tuning)
model = train(model, tokenizer, dataset)
print("\n✅ Training complete!")

---
## 🤖 Phase 2: The Agentic Loop

The agent includes:
- **JSON Extraction**: Robust parsing with markdown code block handling
- **Self-Correction**: 3-retry loop with error context injection
- **Mock Execution**: Simulated system responses

In [ ]:
from unsloth import FastLanguageModel

# Optimize model for inference (disables training-only layers)
FastLanguageModel.for_inference(model)
print("✅ Model ready for inference")

In [ ]:
from src.agent import TriageAgent

# Initialize the triage agent
agent = TriageAgent(model=model, tokenizer=tokenizer)
print("✅ TriageAgent initialized")
print(f"   Max Retries: {config.MAX_RETRIES}")
print(f"   Temperature: {config.GENERATION_TEMPERATURE}")

In [ ]:
# ═══════════════════════════════════════════════════════════════════
#  DEMO: Emergency Case
# ═══════════════════════════════════════════════════════════════════
test_query = """65yo male, crushing chest pain radiating to jaw and left arm.
Started 20 minutes ago. Diaphoretic, nauseous. History of hypertension.
Location: 789 Pine Street."""

print("📋 INPUT Query:")
print("-" * 60)
print(test_query)
print("-" * 60)

# Run the agent
output, response, metadata = agent.run(test_query)

print("\n📤 OUTPUT:")
print("-" * 60)
print(f"Tool Called: {output.tool if output else 'FAILED'}")
print(f"Arguments:   {output.arguments.model_dump() if output else 'N/A'}")
print(f"\n💬 Mock Response: {response}")
print(f"\n🔄 Attempts: {metadata['attempts']} (max: {config.MAX_RETRIES})")
if metadata.get('errors'):
    print(f"⚠️  Errors encountered: {metadata['errors']}")

### 2.1 Convenience Function: `run_triage_agent(query)`

For easier integration, use the standalone function:

In [ ]:
from src.agent import run_triage_agent

# Quick inference with pre-loaded model
result = run_triage_agent(
    query="45yo female with severe abdominal pain, nausea. Location: 123 Main St.",
    model=model,
    tokenizer=tokenizer
)

print("Result:", result["output"])
print("Mock Response:", result["response"])

---
## 📊 Phase 3: Evaluation

Testing against a held-out dataset with metrics:
- **JSON Validity**: % of outputs that parse correctly
- **Routing Accuracy**: % of correct tool selections

In [ ]:
from src.evaluator import load_test_data, evaluate, print_report

# Load held-out test cases
test_data = load_test_data()
print(f"📋 Loaded {len(test_data)} test cases")

In [ ]:
# Run evaluation (this may take a few minutes)
results = evaluate(agent, test_data, verbose=True)

In [ ]:
# Print final evaluation report
print_report(results)

---
## 📝 Summary

### Results

| Metric | Result |
|--------|--------|
| **Model** | Qwen2.5-1.5B (4-bit quantized) |
| **Training Examples** | 100 |
| **JSON Validity** | See report above |
| **Routing Accuracy** | See report above |

### Task Requirements Checklist

| Requirement | Status |
|-------------|--------|
| Runs on Colab Free Tier (T4 GPU) | ✅ |
| Model size ≤ 3B parameters | ✅ (1.5B) |
| Outputs valid JSON | ✅ |
| Handles 3 tools exactly | ✅ |
| Self-correction/error handling | ✅ |
| Evaluation on 10+ test cases | ✅ |
| Zero-cost execution | ✅ |

### Key Implementation Highlights

1. **Efficient Fine-Tuning**: LoRA adapters with 4-bit quantization fit in T4 VRAM
2. **Robust Parsing**: Handles markdown code blocks, partial JSON, and edge cases
3. **Self-Correction**: Error context injection enables recovery from failures
4. **Strict Validation**: Pydantic schemas ensure exact JSON structure